In [ ]:
!pip uninstall opencv-contrib-python
!pip uninstall opencv-python
!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17

In [ ]:
from skimage.transform import warp, AffineTransform
from skimage.filters import gaussian
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def extractSIFT(image):
  sift = cv2.xfeatures2d.SIFT_create(5)
  keypoints, descriptors = sift.detectAndCompute(image, None)
  return descriptors

In [ ]:
def extractSURF(image):
   surf = cv2.xfeatures2d.SURF_create(5)
   keypoints, descriptors = surf.detectAndCompute(image, None)
   return descriptors

In [ ]:
!mkdir '/content/images'
!unzip '/content/drive/MyDrive/rice disease/Images.zip' -d '/content/images'

Streaming output truncated to the last 5000 lines.
  inflating: /content/images/id_c8sl86v71p.jpg  
  inflating: /content/images/id_c8sl86v71p_rgn.jpg  
  inflating: /content/images/id_c8zqlaoscm.jpg  
  inflating: /content/images/id_c8zqlaoscm_rgn.jpg  
  inflating: /content/images/id_c8zzji9xf7.jpg  
  inflating: /content/images/id_c8zzji9xf7_rgn.jpg  
  inflating: /content/images/id_c996aefk2b.jpg  
  inflating: /content/images/id_c996aefk2b_rgn.jpg  
  inflating: /content/images/id_ca1ulb7phu.jpg  
  inflating: /content/images/id_ca1ulb7phu_rgn.jpg  
  inflating: /content/images/id_cai4ccm9qm.jpg  
  inflating: /content/images/id_cai4ccm9qm_rgn.jpg  
  inflating: /content/images/id_carqo16fzo.jpg  
  inflating: /content/images/id_carqo16fzo_rgn.jpg  
  inflating: /content/images/id_cawcl2dlbx.jpg  
  inflating: /content/images/id_cawcl2dlbx_rgn.jpg  
  inflating: /content/images/id_cb2ltrvv0p.jpg  
  inflating: /content/images/id_cb2ltrvv0p_rgn.jpg  
  inflating: /content/images/id

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/rice disease/Train.csv',index_col=False)

In [ ]:
train_df.head()

Image_id  Label
0      id_004wknd7qd.jpg  blast
1  id_004wknd7qd_rgn.jpg  blast
2      id_005sitfgr2.jpg  brown
3  id_005sitfgr2_rgn.jpg  brown
4      id_00stp9t6m6.jpg  blast

In [ ]:
index = [idx for idx in range(5340) if idx%2 !=0]

In [ ]:
train_df_new = train_df.drop(index,axis=0).reset_index(drop=True)

In [ ]:
train , validation = train_test_split(train_df_new,test_size=0.2,stratify=train_df_new['Label'],random_state=42,)

In [ ]:
idg = ImageDataGenerator(rotation_range=40,horizontal_flip=True,vertical_flip=True)


In [ ]:
train_dfgn = idg.flow_from_dataframe(train,'/content/images',x_col = 'Image_id',y_col='Label',batch_size=1,target_size=(224,224),class_mode='sparse',data_format='channels_last')

Found 2136 validated image filenames belonging to 3 classes.


In [ ]:
train_dfgn.class_indices

{'blast': 0, 'brown': 1, 'healthy': 2}

In [ ]:
def extractFeatures(dataFrame,extractor,test=False):
  imageIds = dataFrame['Image_id']
  if not test:
    images_label = dataFrame['Label']
  dataset_df = pd.DataFrame()
  for imageId in imageIds:
    image = cv2.imread(f'/content/images/{imageId}')
    image = cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY),(224,224))
    #rgnid = imageId.split('.jpg')[0]+'_rgn.jpg'
    #image_rgn = cv2.imread(f'/content/images/{rgnid}')
    #image_rgn = cv2.resize(cv2.cvtColor(image_rgn, cv2.COLOR_BGR2RGB),(224,224))
    features_sift = extractor(image).flatten()
    #features_sift_rgn = extractSIFT(image_rgn).flatten()
    #features_surf = extractSURF(image).flatten()
    #final_features = np.concatenate([features_sift,features_sift_rgn])

    dataset_df = dataset_df.append(pd.Series(features_sift), ignore_index=True)
  if not test  :
    return pd.concat([imageIds,dataset_df,images_label])
  else: return dataset_df

In [ ]:
def extractFeaturesAugmented():
  dataset_df = pd.DataFrame()
  labels = pd.DataFrame()
  for _ in range(len(train)):
    x,y = train_dfgn.next()
    image = cv2.normalize(x[0], None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    features_sift = extractSIFT(image).flatten()
    dataset_df = dataset_df.append(pd.Series(features_sift), ignore_index=True)
    labels = labels.append(pd.Series(int(y[0])),ignore_index=True)
  return pd.concat([dataset_df,labels],axis=1)

In [ ]:
def imputeNullValues(data, ratio=50):
    '''
    this function accepts a dataframe which contains NaN values and
    the goal is impute these values with various techniques ==> mean()
    '''
    def clean_column(series):
        series = pd.to_numeric(series, downcast="float")
        avg = series.mean()
        return series.fillna(avg)

    for i in range((data.shape)[1]):
        if (data[i].isna().sum() / (data.shape)[0]) * 100 >= ratio:
            data.drop(data[[i]], axis=1, inplace=True)
        else:
            data[i] = clean_column(data[i])

    return data

In [ ]:
training_features,y = extractFeatures(train)

In [ ]:
train

Image_id    Label
339   id_4q1gs6sy07.jpg    brown
57    id_0rt7042emf.jpg    blast
2485  id_xhh2e7a3id.jpg    blast
6     id_01z6i8am9b.jpg    blast
2093  id_sae50ick1t.jpg    brown
...                 ...      ...
925   id_cfz0tbjaec.jpg    blast
2379  id_w02anaqd5p.jpg    blast
1491  id_jy0g9jz7q2.jpg    blast
2085  id_s6dqnz9oa4.jpg    blast
1725  id_n7wctqkqyi.jpg  healthy

[2136 rows x 2 columns]

In [ ]:
training_features_aug = extractFeaturesAugmented()

In [ ]:
labelsAug = pd.DataFrame(np.int32(training_features_aug.iloc[:,-1]))

In [ ]:
labelsNormal

339     1
57      0
2485    0
6       0
2093    1
       ..
925     0
2379    0
1491    0
2085    0
1725    2
Name: Label, Length: 2136, dtype: int64

In [ ]:
labelsNormal = y.map(train_dfgn.class_indices)

In [ ]:
labels_concat = pd.concat([labelsNormal,labelsAug],axis=0).reset_index(drop=True)

In [ ]:
labels_concat.columns=['Label']

In [ ]:
training_features_aug = training_features_aug.drop(training_features_aug.iloc[:,-1],axis=1)

In [ ]:
training_features_df = pd.concat([training_features,training_features_aug],axis=0)

In [ ]:
labels_concat

Label
0         1
1         0
2         0
3         0
4         1
...     ...
4267      0
4268      1
4269      1
4270      0
4271      0

[4272 rows x 1 columns]

In [ ]:
from sklearn.utils import shuffle


In [ ]:
training_features_df = pd.concat([training_features_df.reset_index(drop=True),labels_concat.reset_index(drop=True)],axis=1)

In [ ]:
training_features_df = imputeNullValues(training_features_df)

In [ ]:
training_features_df = shuffle(training_features_df)

In [ ]:
training_features_df.to_csv('/content/drive/MyDrive/rice disease/train_sift_v2.csv',index=False)

In [ ]:
validation

In [ ]:
val_features,label_val = extractFeatures(validation)

In [ ]:
label_val = label_val.map(train_dfgn.class_indices)

In [ ]:
val_features = pd.concat([val_features.reset_index(drop=True),label_val.reset_index(drop=True)],axis=1)

In [ ]:
val_features = imputeNullValues(val_features)

In [ ]:
val_features.to_csv('/content/drive/MyDrive/rice disease/val_sift_v2.csv',index=False)

In [ ]:
test  = pd.read_csv('/content/SampleSubmission.csv')

In [ ]:
features_df_test = extractFeatures(test,True)

In [ ]:
features_df_test = imputeNullValues(features_df_test)

In [ ]:
features_df_test.to_csv('/content/drive/MyDrive/rice disease/test_sift_v2.csv',index=False)